# Block 6 Exercise 1: Non-Linear Classification

## MNIST Data
We return to the MNIST data set on handwritten digits to compare non-linear classification algorithms ...   

In [4]:
#imports 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

In [5]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)


In [4]:
#the full MNIST data set contains 70k samples of digits 0-9 as 28*28 gray scale images (represented as 784 dim vectors)
np.shape(X)

(70000, 784)

In [5]:
X.min()

0.0

In [6]:
#look at max/min value in the data
X.max()

255.0

### E1.1: Cross-Validation and Support Vector Machines
Train and optimize  C-SVM classifier on MNIST (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
* use a RBF kernel
* use *random search* with cross-validation to find the best settings for *gamma* and *C* (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [8]:
# prepare data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
# train and test SVC with RBF kernel
svm = SVC() 
svm.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
svm.score(X_test,y_test)

0.9801142857142857

In [11]:
# optimization
opti_svm = SVC()
random_search = RandomizedSearchCV(opti_svm, {'C':[0.5, 1.0, 2.0], 'gamma':['scale', 'auto', 0.5, 1, 2, 5]}, n_jobs=-1)

In [12]:
search_results = random_search.fit(X_train[:1000],y_train[:1000]) # takes ages with the whole dataset

In [13]:
print(search_results.best_params_)
print(search_results.best_score_)

{'gamma': 'scale', 'C': 0.5}
0.8859999999999999


In [14]:
#train again with C=0.5
svm = SVC(C=0.5)
svm.fit(X_train,y_train)

SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
svm.score(X_test,y_test)

0.9738285714285714

### E1.2: Pipelines and simple Neural Networks
Split the MNIST data into  train- and test-sets and then train and evaluate a simple Multi Layer Perceptron (MLP) network. Since the non-linear activation functions of MLPs are sensitive to the scaling on the input (recall the *sigmoid* function), we need to scale all input values to [0,1] 

* combine all steps of your training in a SKL pipeline (https://scikit-learn.org/stable/modules/compose.html#pipeline)
* use a SKL-scaler to scale the data (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* MLP Parameters: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
    * use a *SGD* solver
    * use *tanh* as activation function
    * compare networks with 1, 2 and 3 layers, use different numbers of neurons per layer
    * adjust training parameters *alpha* (regularization) and *learning rate* - how sensitive is the model to these parameters?
    * Hint: do not change all parameters at the same time, split into several experiments
* How hard is it to find the best parameters? How many experiments would you need to find the best parameters?
    


In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [8]:
# prepare data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [64]:
pipeline = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(256,256,256), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.1, max_iter=300))

In [65]:
pipeline.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=0.0001,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(256, 256, 256),
                               learning_rate='constant', learning_rate_init=0.1,
                               max_fun=15000, max_iter=300, momentum=0.9,
                               n_iter_no_change=10, nesterovs_momentum=True,
                               power_t=0.5, random_state=None, shuffle=True,
                               solver='sgd', tol=0.0001,
                               validation_fraction=0.1, verbose=False,
                               warm_start=False))],
         verbose=False)

In [66]:
pipeline.score(X_test,y_test)

0.9711428571428572

As there are so many parameters, it's very hard to find the best ones. Here are some of my trys:

MLPClassifier(hidden_layer_sizes=(64), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.001)

Score: 0.9527428571428571

MLPClassifier(hidden_layer_sizes=(64), activation='tanh', solver='sgd', alpha=0.001, learning_rate='constant', learning_rate_init=0.001, max_iter=300)

Score: 0.9552

MLPClassifier(hidden_layer_sizes=(64), activation='tanh', solver='sgd', alpha=0.00001, learning_rate='constant', learning_rate_init=0.001, max_iter=300)

Score: 0.9569714285714286

* alpha doesn't have a big impact

pipeline = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(64,64,64), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.001, max_iter=300))

Score: 0.9529714285714286

* more layers don't help

MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.001, max_iter=300)

Score: 0.9592

* more neurons help a little

MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.01, max_iter=300)

Score: 0.9613142857142857

* higher learning_rate is better

MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.1, max_iter=300))

0.9672571428571428

MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='adaptive', learning_rate_init=0.1, max_iter=300)

Score: 0.9672

* same result with adaptive learning rate

MLPClassifier(hidden_layer_sizes=(128,128,128), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='invscaling', learning_rate_init=0.1, max_iter=300, power_t=0.1)

Score: 0.9649714285714286

* Best result:
MLPClassifier(hidden_layer_sizes=(256,256,256), activation='tanh', solver='sgd', alpha=0.0001, learning_rate='constant', learning_rate_init=0.1, max_iter=300)

Score: 0.9711428571428572